<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#SGDClassifier" data-toc-modified-id="SGDClassifier-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>SGDClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd

import re

import nltk

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score

from sklearn.utils import shuffle

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')


In [2]:
comments = pd.read_csv('/datasets/toxic_comments.csv')
comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
display(comments.head(5))

print('-----------------------------------------------------------------')
comments.info()

print('-----------------------------------------------------------------')

print('Дубликатов -', comments.duplicated().sum())

print('-----------------------------------------------------------------')

print('Пропусков:')
display(comments.isna().sum())

print('-----------------------------------------------------------------')

print('Соотношение в целевом признаке:')
display(comments.toxic.value_counts(normalize=True))

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


-----------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
-----------------------------------------------------------------
Дубликатов - 0
-----------------------------------------------------------------
Пропусков:


Unnamed: 0    0
text          0
toxic         0
dtype: int64

-----------------------------------------------------------------
Соотношение в целевом признаке:


0    0.898388
1    0.101612
Name: toxic, dtype: float64

В таблице 159 292 объектов. Пропусков нет, явных дубликатов нет
Тексты комментариев на английском, есть лишние знаки типа "\nMore\n
В целевом признаке 90% объектов отрицательного класса, то есть в дальнейшем нужно будет учесть это
Необходимо избавиться от столбца Unnamed, так как он фактически дублирует индексы

In [5]:
#удаляю столбец Unnamed:
comments = comments.drop(['Unnamed: 0'], axis=1)

In [6]:
#ввожу функцию очищения текстов постов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [7]:
%%time
#очищаю тексты постов:
comments['text'] = comments['text'].apply(clear_text) 

CPU times: user 5.17 s, sys: 26.1 ms, total: 5.19 s
Wall time: 5.23 s


In [8]:
comments.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


In [9]:
#ввожу функцию РОS-тэгирования слов:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

#ввожу функцию леммализации тектов постов:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [10]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [11]:
%%time

#леммализирую тексты постов:
comments['text'] = comments['text'].apply(lemm_text) 

CPU times: user 23min 29s, sys: 2min 5s, total: 25min 35s
Wall time: 25min 47s


In [12]:
comments.head()

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0


In [17]:
#разделяю выборки в соотношении 80/10/10:
features = comments.drop(['toxic'], axis=1) 
target = comments.toxic

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.2, 
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=.5,
                                                                            random_state=12345)

In [18]:
#смотрю размеры выборок:
for i in [features_train, target_train, features_valid, target_valid, features_test, target_test]:
    print(i.shape)

(127433, 1)
(127433,)
(15929, 1)
(15929,)
(15930, 1)
(15930,)


In [19]:
#смотрю соотношение 1/0 в выборках на примере target_train:
indices_1 = [i for i,x in enumerate(target_train) if x == 1]
count_1 = len(indices_1)

indices_0 = [i for i,x in enumerate(target_train) if x == 0]
count_0 = len(indices_0)

print('Доля значений 1 в тренировочной выборке:', len(indices_1) / (len(indices_1) + len(indices_0)))

Доля значений 1 в тренировочной выборке: 0.10166911239631807


In [20]:
#уменьшаю кол-во 0 в выборках train:

comments_train = comments.iloc[target_train.index]
target_train_0 = comments_train[comments_train['toxic'] == 0]['toxic']
target_train_1 = comments_train[comments_train['toxic'] == 1]['toxic']


target_train_0_resample = target_train_0.sample(target_train_1.shape[0], random_state=12345)
target_train_resample = pd.concat([target_train_0_resample, target_train_1])

features_train_resample = comments.iloc[target_train_resample.index]

features_train_resample, target_train_resample = shuffle(features_train_resample,
                                                         target_train_resample,
                                                         random_state=12345)

features_train_resample = features_train_resample.text 

print('Соотношение 1/0 в тренировочной выборке:')
print(target_train_resample.value_counts(normalize=True))
print()
print(features_train_resample.shape)
print(target_train_resample.shape)

Соотношение 1/0 в тренировочной выборке:
0    0.5
1    0.5
Name: toxic, dtype: float64

(25912,)
(25912,)


Выводы по подготовка данных:
При подготовке данных:
- Удалила ненужный столбец
- Очистила тексты комментариев от ненужных знаков, леммализировала, убрала стоп-слова
- Сбалансировала данные в целевом признаке

## Обучение

### LogisticRegression

In [22]:
features_train = features_train.text

In [23]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200]),
              'lr__class_weight': (['balanced'])} 
                                                  
gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

print('F1 логистической регрессии =', round(lr_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
lr_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 логистической регрессии на валидации =', round(lr_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
lr_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 логистической регрессии =', round(lr_test_f1,2))

F1 логистической регрессии = 0.76
при параметрах {'lr__C': 5, 'lr__class_weight': 'balanced', 'lr__max_iter': 200, 'lr__random_state': 12345, 'lr__solver': 'lbfgs'}

F1 логистической регрессии на валидации = 0.76

финальный F1 логистической регрессии = 0.76
CPU times: user 15min 34s, sys: 13min 37s, total: 29min 12s
Wall time: 29min 25s


### DecisionTreeClassifier

In [24]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("dtc", DecisionTreeClassifier())])
    
parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__random_state': ([12345]), 
              'dtc__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train_resample, target_train_resample)

mts = gscv.cv_results_['mean_test_score']
dtc_train_f1 = max(mts)

print('F1 дерева решений =', round(dtc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
dtc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 дерева решений на валидации =', round(dtc_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
dtc_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 дерева решений =', round(dtc_test_f1,2))

F1 дерева решений = 0.71
при параметрах {'dtc__class_weight': 'balanced', 'dtc__max_depth': 24, 'dtc__random_state': 12345}

F1 дерева решений на валидации = 0.62

финальный F1 дерева решений = 0.61
CPU times: user 2min 31s, sys: 473 ms, total: 2min 31s
Wall time: 2min 33s


### CatBoostClassifier

In [25]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("cbc", CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200]),
              'cbc__class_weights':([(1, 1), (1, 11)])} 

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train_resample, target_train_resample)

mts = gscv.cv_results_['mean_test_score']
cbc_train_f1 = max(mts)

print('F1 CatBoostClassifier =', round(cbc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
cbc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 CatBoostClassifier на валидации =', round(cbc_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
cbc_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 CatBoostClassifier =', round(cbc_test_f1,2))

F1 CatBoostClassifier = 0.87
при параметрах {'cbc__class_weights': (1, 1), 'cbc__iterations': 200, 'cbc__verbose': False}

F1 CatBoostClassifier на валидации = 0.72

финальный F1 CatBoostClassifier = 0.72
CPU times: user 11min 15s, sys: 5.51 s, total: 11min 21s
Wall time: 11min 31s


### RandomForestClassifier

In [26]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("rfc", RandomForestClassifier())])
    
parameters = {'rfc__n_estimators': ([x for x in range(10, 30)]),
              'rfc__random_state': ([12345]),
              'rfc__max_depth': ([x for x in range(1, 10)]),
              'rfc__criterion': (['entropy']),
              'rfc__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train_resample, target_train_resample)

mts = gscv.cv_results_['mean_test_score']
rfc_train_f1 = max(mts)

print('F1 случайного леса =', round(rfc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
rfc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 случайного леса на валидации =', round(rfc_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
rfc_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 случайного леса =', round(rfc_test_f1,2))

F1 случайного леса = 0.77
при параметрах {'rfc__class_weight': 'balanced', 'rfc__criterion': 'entropy', 'rfc__max_depth': 9, 'rfc__n_estimators': 29, 'rfc__random_state': 12345}

F1 случайного леса на валидации = 0.33

финальный F1 случайного леса = 0.33
CPU times: user 12min 48s, sys: 2.03 s, total: 12min 50s
Wall time: 12min 56s


### SGDClassifier

In [27]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("clf", SGDClassifier())])
    
parameters = {'clf__loss': ('hinge', 'log', 'modified_huber'),
              'clf__learning_rate': ('constant', 'optimal', 'invscaling', 'adaptive'),
              'clf__eta0': (.01, .05, .1, .5),
              'clf__random_state': ([12345]),
              'clf__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train_resample, target_train_resample)

mts = gscv.cv_results_['mean_test_score']
sgdc_train_f1 = max(mts)

print('F1 SGDClassifier =', round(sgdc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
sgdc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 SGDClassifier на валидации =', round(sgdc_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
sgdc_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 SGDClassifier =', round(sgdc_test_f1,2))

F1 SGDClassifier = 0.89
при параметрах {'clf__class_weight': 'balanced', 'clf__eta0': 0.01, 'clf__learning_rate': 'adaptive', 'clf__loss': 'modified_huber', 'clf__random_state': 12345}

F1 SGDClassifier на валидации = 0.69

финальный F1 SGDClassifier = 0.7
CPU times: user 3min 46s, sys: 14.2 s, total: 4min
Wall time: 4min 6s


## Выводы

In [28]:
#создаю сводную таблицу по показателям F1, времени обучения модели и времени предсказания модели:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'CatBoostClassifier',
         'RandomForestClassifier',
         'SGDClassifier'
        ]

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    cbc_train_f1,
                                    rfc_train_f1,
                                    sgdc_train_f1],
        
        'F1 на валидационной выборке': [lr_valid_f1,
                                        dtc_valid_f1,
                                        cbc_valid_f1,
                                        rfc_valid_f1,
                                        sgdc_valid_f1],
        
        'F1 на тестовой выборке (финальный)': [lr_test_f1,
                                               dtc_test_f1,
                                               cbc_test_f1,
                                               rfc_test_f1,
                                               sgdc_test_f1]}

f1_data = pd.DataFrame(data=data, index=index)

f1_data.sort_values(by='F1 на тестовой выборке (финальный)', ascending=False)

,F1 на обучающей выборке,F1 на валидационной выборке,F1 на тестовой выборке (финальный)
LogisticRegression,0.756144,0.764114,0.764640
CatBoostClassifier,0.872809,0.722311,0.722698
SGDClassifier,0.891635,0.694762,0.697514
DecisionTreeClassifier,0.712648,0.622162,0.612903
RandomForestClassifier,0.774037,0.334877,0.332890


В проекте:
Загрузила данные и провела их предобработку - удаление лишних данных, очистку текстов, лемматизацию.

Обучила 5 моделей с разными гиперпараметрами и выборками и проверила их на тестовой выборке.

Выбрала лучшую модель по показателю F1.
Худшими показателями среди моделей обладают RandomForestClassifier и DecisionTreeClassifier, так как дали наименьшее F1.

Наилучшей моделью стала LogisticRegression, которая на тестировании показала F1 = 0.76. Поскольку требовалось найти модель классификации комментариев на позитивные и негативные со значением метрики качества F1 >= 0.75, рекомендовать могу LogisticRegression.